In [ ]:
import os
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import pandas as pd
import numpy as np
from scipy import stats
from scipy.spatial import distance
from matplotlib import gridspec
from sklearn.preprocessing import normalize
import pandas as pd
import ast


import caiman as cm
from caiman.source_extraction import cnmf
from caiman.utils.utils import download_demo
from caiman.utils.visualization import inspect_correlation_pnr
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import params as params
from caiman.source_extraction.cnmf.deconvolution import constrained_foopsi
from caiman.source_extraction.cnmf.deconvolution import constrained_oasisAR2

#plotting params
mpl.rcParams['axes.facecolor'] = 'white'
mpl.rcParams['axes.edgecolor'] = 'black'
mpl.rcParams['axes.linewidth'] = '0.5'
mpl.rcParams['axes.labelsize'] = '8'
mpl.rcParams['axes.labelcolor'] = 'black'

mpl.rcParams['xtick.color'] = 'black'
mpl.rcParams['xtick.labelsize'] = '4'
mpl.rcParams['ytick.labelsize'] = '4'
mpl.rcParams['ytick.color'] = 'black'

In [ ]:
folder = r'C:\Users\fr87_\Documents\Python code\Input folder\H2224'
file='H2224_SE28_TRA.csv'


all_traces_dlc = pd.read_csv(os.path.join(folder, file))


In [ ]:
#remove rejected cells 
all_traces_dlc.T[all_traces_dlc.T[0]!=' rejected'].T
all_traces_dlc=all_traces_dlc.T[all_traces_dlc.T[0]!=' rejected'].T[1:]

In [ ]:
#remove NaNs and covert them to zero (single session)
all_traces_dlc.rename(columns={' ':'Time'}, inplace=True)
all_traces_dlc = all_traces_dlc.replace(' nan', np.nan)
all_traces_dlc=all_traces_dlc.fillna(0)
all_traces_dlc=all_traces_dlc.reset_index(drop=True)
all_traces_dlc

In [ ]:
raw_traces = all_traces_dlc.reset_index(drop=True)
be_a_cell=[col for col in raw_traces.columns if col.startswith(' C')]
raw_traces=raw_traces[be_a_cell]
cells = raw_traces.columns
#be_a_cell

In [ ]:
# #Params to investigate:
bl=None
c1=None
g=None
sn=15
p=1
method_deconvolution='oasis'
bas_nonneg=True
noise_range=[0, 50]
noise_method='logmexp'
lags=5
fudge_factor=.98
verbosity=False
solvers=None
optimize_g=100
s_min=0.2

#--------------------------------------------------------------------------------------------------------------------------------------------

trace_dlc = np.array(raw_traces.values.astype(float)).transpose()[:]
cells_tv = cells[:]
all_events = []
for cell in range(5)):#(len(trace_dlc)):
    
    raw_trace = trace_dlc[cell]
#     raw_trace = np.array(raw_trace)/max(np.array(raw_trace)) #If we want traces between 0 and 1 (OPTIONAL)

    #OASIS function run through
    events = constrained_foopsi(raw_trace, 
                                bl=bl,  
                                c1=c1, 
                                g=g,  
                                sn=sn, 
                                p=p, 
                                method_deconvolution=method_deconvolution, 
                                bas_nonneg=bas_nonneg,
                                noise_range=noise_range,
                                noise_method=noise_method,
                                lags=lags,
                                fudge_factor=fudge_factor,
                                verbosity=verbosity,
                                solvers=solvers,
                                optimize_g=optimize_g,
                                s_min=s_min)
    
    
    all_events.append(events[5])
    #Viualising data
    fig, axs = plt.subplots(3, 1, figsize=(10,3), dpi=400, facecolor='w', edgecolor='k')
    gs = gridspec.GridSpec(3, 1, height_ratios=[1,1,1]) 
    fig.suptitle(cells_tv[cell], fontsize=10)

    #Raw traces
    axs[0] = plt.subplot(gs[0])
    axs[0] = plt.plot(raw_trace, c='k', linewidth=1)

    #OASIS noise consideration
    axs[1] = plt.subplot(gs[1])
    axs[1] = plt.plot(events[0], c='b', linewidth=1)
    
    #Events plotted on raw trace
    axs[2] = plt.subplot(gs[2])
    axs[2] = plt.plot(raw_trace, c='k', alpha=0.1, linewidth=1)
    axs[2] = plt.scatter(np.arange(0,len(events[5]),1),events[5], c='r', s=5)
   
    plt.subplots_adjust(wspace=0, hspace=0)
    plt.show()
    


In [ ]:
need_higher_sn=[
try_lower=[

In [ ]:

need_higher_sn=[' C' + i for i in need_higher_sn.split(sep='.')]
try_lower=[' C' + i for i in try_lower.split(sep='.')]

a = ['x' for x in need_higher_sn if x in try_lower]

if a != []:
    print('Warning! The lists are overlapping')
else: 
    print('Ready to go on')

all_events_copy=all_events.copy()
all_events_copy=np.transpose(all_events_copy)



In [ ]:
#DataFrame with the detected events

all_events_df=pd.DataFrame(data=all_events_copy, columns=be_a_cell)


In [ ]:
all_events_df

In [ ]:
#change the parameters for the subset "need higher SN"
sn=25
p=1
method_deconvolution='oasis'
bas_nonneg=True
noise_range=[0, 100]
noise_method='logmexp'
lags=5
fudge_factor=.98
verbosity=False
solvers=None
optimize_g=10
s_min=0.4


if need_higher_sn != [' C']:
    trace_dlc = np.array(raw_traces[need_higher_sn].values.astype(float)).transpose()[:]
    cells = raw_traces[need_higher_sn].columns

    cells_tv = cells[:]
    all_events = []

    for cell in range(len(trace_dlc)):

        raw_trace = trace_dlc[cell]
    #     raw_trace = np.array(raw_trace)/max(np.array(raw_trace)) #If we want traces between 0 and 1 (OPTIONAL)

        #OASIS function run through
        events = constrained_foopsi(raw_trace, 
                                    bl=bl,  
                                    c1=c1, 
                                    g=g,  
                                    sn=sn, 
                                    p=p, 
                                    method_deconvolution=method_deconvolution, 
                                    bas_nonneg=bas_nonneg,
                                    noise_range=noise_range,
                                    noise_method=noise_method,
                                    lags=lags,
                                    fudge_factor=fudge_factor,
                                    verbosity=verbosity,
                                    solvers=solvers,
                                    optimize_g=optimize_g,
                                    s_min=s_min)


        all_events.append(events[5])
        #Viualising data
        fig, axs = plt.subplots(3, 1, figsize=(10,3), dpi=400, facecolor='w', edgecolor='k')
        gs = gridspec.GridSpec(3, 1, height_ratios=[1,1,1]) 
        fig.suptitle(cells_tv[cell], fontsize=10)

        #Raw traces
        axs[0] = plt.subplot(gs[0])
        axs[0] = plt.plot(raw_trace, c='k', linewidth=1)

        #OASIS noise consideration
        axs[1] = plt.subplot(gs[1])
        axs[1] = plt.plot(events[0], c='b', linewidth=1)

        #Events plotted on raw trace
        axs[2] = plt.subplot(gs[2])
        axs[2] = plt.plot(raw_trace, c='k', alpha=0.1, linewidth=1)
        axs[2] = plt.scatter(np.arange(0,len(events[5]),1),events[5], c='r', s=5)

        plt.subplots_adjust(wspace=0, hspace=0)
        plt.show()

    #DataFrame with the events for noisier cells only    

    all_events_df_higher_sn=pd.DataFrame(data=np.transpose(all_events), columns=need_higher_sn)

elif need_higher_sn == [' C']:
    print('No cells to re-analyse')

In [ ]:
#change the parameters for the subset "try lower"

sn=10
p=1
method_deconvolution='oasis'
bas_nonneg=True
noise_range=[0, 200]
noise_method='logmexp'
lags=5
fudge_factor=.98
verbosity=False
solvers=None
optimize_g=5
s_min=0.4

trace_dlc = np.array(raw_traces[try_lower].values.astype(float)).transpose()[:]
cells = raw_traces[try_lower].columns

cells_tv = cells[:]
all_events = []

for cell in range(len(trace_dlc)):
    
    raw_trace = trace_dlc[cell]
#     raw_trace = np.array(raw_trace)/max(np.array(raw_trace)) #If we want traces between 0 and 1 (OPTIONAL)

    #OASIS function run through
    events = constrained_foopsi(raw_trace, 
                                bl=bl,  
                                c1=c1, 
                                g=g,  
                                sn=sn, 
                                p=p, 
                                method_deconvolution=method_deconvolution, 
                                bas_nonneg=bas_nonneg,
                                noise_range=noise_range,
                                noise_method=noise_method,
                                lags=lags,
                                fudge_factor=fudge_factor,
                                verbosity=verbosity,
                                solvers=solvers,
                                optimize_g=optimize_g,
                                s_min=s_min)
    
    
    all_events.append(events[5])
    #Viualising data
    fig, axs = plt.subplots(3, 1, figsize=(10,3), dpi=400, facecolor='w', edgecolor='k')
    gs = gridspec.GridSpec(3, 1, height_ratios=[1,1,1]) 
    fig.suptitle(cells_tv[cell], fontsize=10)

    #Raw traces
    axs[0] = plt.subplot(gs[0])
    axs[0] = plt.plot(raw_trace, c='k', linewidth=1)

    #OASIS noise consideration
    axs[1] = plt.subplot(gs[1])
    axs[1] = plt.plot(events[0], c='b', linewidth=1)
    
    #Events plotted on raw trace
    axs[2] = plt.subplot(gs[2])
    axs[2] = plt.plot(raw_trace, c='k', alpha=0.1, linewidth=1)
    axs[2] = plt.scatter(np.arange(0,len(events[5]),1),events[5], c='r', s=5)
   
    plt.subplots_adjust(wspace=0, hspace=0)
    plt.show()
    
#DataFrame containing the event traces for cells needing a lower sn parameter
    
all_events_df_lower_sn=pd.DataFrame(data=np.transpose(all_events), columns=try_lower)


In [ ]:
def overwrite_events(original_df, dfs_to_be_written):
    
    """
    This function takes as argument two DataFrames and overwrites the common columns from df_to_be_written. 
    The resulting DataFrame cointains the columns from df_to_be_written and the other columns from original_df
    
    original_df:       original dataframes. Columns with the same ID as df_to_be_written will be deleted and substituted with the 
                       corresponding columns from df_to_be_written
                    
    dfs_to_be_written:  a list of dataframes to be written over original_df. They need not to share columns between each other
    
    final_df:          returns the final dataframe
    
    
    e.g. orginal_df :  A  | B  | C  | D
                       a1 | b1 | c1 | d1
                       a2 | b2 | c2 | d2
    
    df_to_be_written:  B  | C  
                       k1 | j1
                       k2 | j2
    
    final_df:          A  | B  | C  | D
                       a1 | k1 | j1 | d1
                       a2 | k2 | j2 | d2   
                       
                       
    """
    columns_list=[]
    
    for df in dfs_to_be_written:
        columns_list=columns_list+list(df.columns)
        
    if [x for x in columns_list if x not in list(set(columns_list))] != []:
        print('The DataFrames you are trying to overwrite are overlapping')
        
    else:      
        df_to_be_written=pd.concat(dfs_to_be_written, axis=1)
        final_df=df_to_be_written.copy()
        other_columns=[i for i in list(original_df.columns) if i not in list(df_to_be_written.columns)]
        final_df[other_columns]=original_df[other_columns]
        return final_df
    
    
all_events_df_4=all_events_df.copy()

all_events_df_3=pd.DataFrame(data=np.transpose(all_events), columns=try_lower)


In [ ]:
all_events_df=overwrite_events(all_events_df, [all_events_df_lower_sn,all_events_df_higher_sn])
print(all_events_df.columns[1:100])

In [ ]:
cell = ' C012'

(all_events_df_4[cell]*(-1)).plot(color='k')

all_events_df[cell].plot(color='green')


In [ ]:
all_events_df=all_events_df.sort_index(axis=1)
all_events_df['Time']=all_traces_dlc['Time']
all_events_df

In [ ]:
output_folder=r'C:\Users\fr87_\Documents\Python code\Output folder'
all_events_df.to_csv(os.path.join(output_folder, file[:10] + 'EVE.csv'))